In [41]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import matplotlib.pyplot as plt
import json

C:\Users\Kiki\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [42]:
## Load data
imdb_genres = pd.read_csv("genres.csv", sep=';') # IMDB genre list to compare identified genres to
movie_titles = pd.read_csv("movie_titles.csv", sep=';', encoding = 'latin')
movies_matched = pd.read_csv("movies_matched.csv")
submissions = pd.read_csv("submissions.csv", sep=';')
submissions = submissions.fillna("")
submissions['all_text'] = submissions["reddit_submission_title"] + " " + submissions["reddit_submission_text"]
test_data = pd.read_csv("test_submissions_simplified.csv")
train_data = pd.read_csv("train_submissions_simplified.csv")

In [43]:
## Load predictions and revert them, then save them

name = 'roberta_lg_mgn_1'
with open(name + "_matched_format_1.json") as json_file:
    all_predictions = json.load(json_file)
    
for key in all_predictions.keys():
    all_predictions[key].pop('positive_genres', 'not found')
    all_predictions[key].pop('positive_keywords', 'not found')
    all_predictions[key].pop('positive_actors','not found')
    all_predictions[key].pop('negative_actors','not found')
    all_predictions[key].pop('negative_keywords', 'not found')
    all_predictions[key].pop('negative_genres', 'not found')
    
predictions_reverted = dict()
for key in all_predictions.keys():
    if 'positive_movies' in all_predictions[key].keys():
        for movie in all_predictions[key]['positive_movies']:
            if not movie in predictions_reverted.keys():
                predictions_reverted[movie] = list()
                predictions_reverted[movie].append(key)
            else:
                predictions_reverted[movie].append(key)
    if 'negative_movies' in all_predictions[key].keys():
        for movie in all_predictions[key]['negative_movies']:
            if not movie in predictions_reverted.keys():
                predictions_reverted[movie] = list()
                predictions_reverted[movie].append(key)
            else:
                predictions_reverted[movie].append(key)
                
with open(name + "_reverted_predictions.json", "w") as json_file:
    json.dump(predictions_reverted, json_file)

In [44]:
# Get rid of unnecessary data in the submissions, prepare them for doc2vec

train_df = train_data[['id', 'text']]
test_df = test_data[['id', 'text']]

train_submissions = list()
for index, row in train_df.iterrows():
    train_submissions.append(TaggedDocument(row['text'].lower().split(), row['id']))
    
test_submissions = list()

for index, row in test_df.iterrows():
    test_submissions.append(TaggedDocument(row['text'].lower().split(), row['id']))

In [45]:
# Load a pre-trained doc2vec model

model = Doc2Vec.load("C:/Users/Kiki/Projects/enwiki_dbow/doc2vec.bin")
"""
Train a doc2vec model on the training data
model = Doc2Vec(min_count=3, window=5, vector_size=50, workers=4, alpha=0.025, min_alpha=0.00025, dm=1)
model.build_vocab(train_submissions)
model.train(train_submissions, total_examples=model.corpus_count, epochs=model.epochs)
"""

'\nTrain a doc2vec model on the training data\nmodel = Doc2Vec(min_count=3, window=5, vector_size=50, workers=4, alpha=0.025, min_alpha=0.00025, dm=1)\nmodel.build_vocab(train_submissions)\nmodel.train(train_submissions, total_examples=model.corpus_count, epochs=model.epochs)\n'

In [51]:
# Infer the sentences and create a new dictionary where for each movie the vectors of the submissions where it appears
# will show up

vectors_reverted = dict()
for key in predictions_reverted.keys():
    for sentence in test_submissions:
        inferred_sentence = model.infer_vector(sentence.words).tolist()
        if sentence.tags in predictions_reverted[key]:
            if key not in vectors_reverted.keys():
                vectors_reverted[key] = list()
            vectors_reverted[key].append(inferred_sentence)

In [52]:
# Create summed vectors
for key in vectors_reverted.keys():
    div = len(vectors_reverted[key])
    vectors_reverted[key] = np.add.reduce(vectors_reverted[key]).tolist() 
    #vectors_reverted[key] = np.mean(vectors_reverted[key], axis=0).tolist()

In [53]:
with open("engwiki_300_summed_roberta_large.json", "w") as json_file:
    json.dump(vectors_reverted, json_file)
    
len(vectors_reverted['tt0083658'])

300

In [54]:
# Load imdb dataset

imdb_data = pd.read_csv("C:/Users/Kiki/Projects/imdb_dataset/imdb_data.csv")

# Turn imdb train data to TaggedDocument

imdb_train = list()

for index, row in imdb_data.iterrows():
    imdb_train.append(TaggedDocument(row['review'].lower().split(), str(index)))

In [55]:
"""
Train a doc2vec model on the training data
"""
model = Doc2Vec(min_count=3, window=5, vector_size=64, workers=4, alpha=0.025, min_alpha=0.00025, dm=1)
model.build_vocab(imdb_train)
model.train(imdb_train, total_examples=model.corpus_count, epochs=model.epochs)

In [59]:
vectors_reverted = dict()
for key in predictions_reverted.keys():
    for sentence in test_submissions:
        inferred_sentence = model.infer_vector(sentence.words).tolist()
        if sentence.tags in predictions_reverted[key]:
            if key not in vectors_reverted.keys():
                vectors_reverted[key] = list()
            vectors_reverted[key].append(inferred_sentence)

In [60]:
# Create summed vectors
for key in vectors_reverted.keys():
    div = len(vectors_reverted[key])
    #vectors_reverted[key] = np.add.reduce(vectors_reverted[key]).tolist() 
    vectors_reverted[key] = np.mean(vectors_reverted[key], axis=0).tolist()

In [61]:
with open("imdbrev_64_averaged_roberta_large.json", "w") as json_file:
    json.dump(vectors_reverted, json_file)
    
len(vectors_reverted['tt0083658'])

64

In [108]:
with open("d2v_embedded_test_data.json", "w") as outfile:
    json.dump(test_data_vectors, outfile)

In [109]:
test_data_vectors

{'5ciw72': [0.2121639996767044,
  -0.10748590528964996,
  0.034874506294727325,
  0.11276333034038544,
  -0.14389768242835999,
  0.2718544602394104,
  -0.017764516174793243,
  0.014193648472428322,
  -0.13076914846897125,
  -0.14520084857940674,
  0.13783550262451172,
  0.19968993961811066,
  -0.05696044862270355,
  -0.058903519064188004,
  0.02776845544576645,
  -0.026487823575735092,
  -0.22365951538085938,
  0.009936262853443623,
  0.10787476599216461,
  0.012934782542288303,
  -0.061244040727615356,
  -0.0017707888036966324,
  0.04296526685357094,
  0.037012141197919846,
  -0.24104568362236023,
  -0.15037491917610168,
  0.24988232553005219,
  0.10565707832574844,
  0.08383750170469284,
  0.06525707989931107,
  0.23021629452705383,
  0.12725047767162323,
  0.11814694851636887,
  -0.0904112383723259,
  -0.09462477266788483,
  0.12607042491436005,
  -0.09123755246400833,
  0.04671115800738335,
  -0.31134748458862305,
  0.09314228594303131,
  0.18606208264827728,
  0.02555549331009388,